In [ ]:
import os
import numpy as np 
import damask

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# some pre-requisites, after `source env/DAMASK.sh`
damask_root = %env DAMASK_ROOT

# 1. input files (geometery, material, load)

## geometry file

In [ ]:
# geometry file (3d periodic grid)
size = np.ones(3)*1e-5
cells = [8,8,8]
N_grains = 4
seeds = damask.seeds.from_random(size,N_grains,cells)
grid = damask.GeomGrid.from_Voronoi_tessellation(cells,size,seeds)
grid.save(f'Polycrystal_{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}')
grid

## material file

In [ ]:
# material file has three sections (__homogenization__, __phase__, __material__)
mat = damask.YAML(homogenization={}, phase={}, material={})

In [ ]:
hom_single = {
    'SX': {                            # (1) name of homogenization, SX->single crystal
        'N_constituents': 1,           # (2) components number
        'mechanical': {'type': 'pass'} # (3) type of homogenization
    }
}

mat.update(homogenization=hom_single) # add to material.yaml file

In [ ]:
phase_alu = {
    'Aluminium':{                                    # (1) name of your phase
        'lattice': 'cF',                             # (2) lattice specific
        'mechanical': {                              # (3) mechanical properties
            'output': ['F', 'P', 'F_e', 'L_p', 'O'], # Your desired output quantities
            'elastic': {},
            'plastic': {}
        }
    }
}

In [ ]:
alu_elast = damask.YAML.load(os.path.join(damask_root,'examples/config/phase/mechanical/elastic/Hooke_Al.yaml'))
alu_plast = damask.YAML.load(os.path.join(damask_root,'examples/config/phase/mechanical/plastic/phenopowerlaw_Al.yaml'))

In [ ]:
phase_alu['Aluminium']['mechanical']['elastic']['type'] = alu_elast['type']
phase_alu['Aluminium']['mechanical']['elastic']['C_11'] = alu_elast['C_11']
phase_alu['Aluminium']['mechanical']['elastic']['C_12'] = alu_elast['C_12']
phase_alu['Aluminium']['mechanical']['elastic']['C_44'] = alu_elast['C_44']
phase_alu['Aluminium']['mechanical']['plastic'] = dict(alu_plast)

mat.update(phase=phase_alu) # add to material.yaml file

In [ ]:
# material section -> grain orientation, components and etc
## generate a simple random texture and add it to material section
rnd = damask.Rotation.from_random(N_grains)

In [ ]:
mat_config = damask.ConfigMaterial(**mat)
mat_config.pop('material') # material_add() requires no 'material' key exsists

In [ ]:
# each material is single crystal (homogenization) and alu (phase)
mat_config = mat_config.material_add(O=rnd,phase='Aluminium',homogenization='SX') 
mat_config.save('material.yaml')

## load file

In [ ]:
## handy snippet to account for exclusiveness of stress/strain BC
def make_P(F,fill=0):
    return [make_P(i,fill) if isinstance(i,list) else\
            fill if i == 'x' else 'x' for i in F]

In [ ]:
# each load file contains (1) solver option, (2) several loadsteps 
load_config = damask.LoadcaseGrid(
    solver={'mechanical':'spectral_basic'}, # (1) solver
    loadstep=[]                             # (2) loadsteps                     
)

In [ ]:
# each load step specifies (1) BC (stress or strain), (2) time discretization, (3) output frequency
## load step 1 (tensile in x) ##
dot_F = [[1.e-3, 0 , 0 ],
         [   0 ,'x', 0 ],
         [   0 , 0 ,'x']]

loadstep = {
    'boundary_conditions':{              # (1) BC
        'mechanical':{                
            'dot_F':dot_F,
            'P':make_P(dot_F)
        }
    },
    'discretization':{'t':10.,'N':40},   # (2) time discretization
    'f_out':4                            # (3) output frequency
}

load_config['loadstep'].append(loadstep)

## load step 2 (tensile in x) ##
dot_F = [[1.e-3, 0 , 0 ],
         [   0 ,'x', 0 ],
         [   0 , 0 ,'x']]

loadstep = {
    'boundary_conditions':{              # (1) BC
        'mechanical':{                
            'dot_F':dot_F,
            'P':make_P(dot_F)
        }
    },
    'discretization':{'t':40.,'N':40},   # (2) time discretization
    'f_out':4                            # (3) output frequency
}

load_config['loadstep'].append(loadstep)
load_config.save('load.yaml')

right now you have a similar input file named as 
- `Polycrystal_{N_grains}_{cells}.vti`
- `material.yaml`
- `load.yaml`

in your folder now, which resemble the ones in `ref/`


# 2. run simulation with DAMASK-grid solver

In [ ]:
fn_vti = f'Polycrystal_{N_grains}_{cells[0]}x{cells[1]}x{cells[2]}.vti'
fn_load = 'load.yaml'
fn_material = 'material.yaml'

In [ ]:
%%time
run_log = damask.util.run(f'DAMASK_grid -g {fn_vti} -l {fn_load}  -m {fn_material}')

In [ ]:
print(''.join(run_log))

# 3. process result and visualize

## basic post-processing

In [ ]:
fn_hdf = '_'.join((fn_vti.split('.')[0], fn_load.split('.')[0], fn_material.split('.')[0]))+'.hdf5'
result = damask.Result(fn_hdf)
result

In [ ]:
# post-processing: (1) add variables, (2) view certain increments, (3) export for paraview
## (1) can act only on certain increments! when you have too large data
result.add_stress_Cauchy()                      # Cauchy stress
result.add_strain()                             # ln(V)    
result.add_equivalent_Mises('sigma')            # von Mises value maybe more representative
result.add_equivalent_Mises('epsilon_V^0.0(F)') # von Mises like strain based on ln(V) 

In [ ]:
## (2) various ways to view
result.view(times=result.times_in_range(5,35))
result.view(increments=-1)

In [ ]:
## (3) various export options
result.export_VTK()
result.export_simulation_setup(target_dir='sim_setup')

## customized analysis

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
grains = [0,1,3]
grid   = damask.GeomGrid.load(fn_vti) # get back my grid
# loop over all increments and store P(1,1) per grain and avg(F(1,1))
data = {g:pd.DataFrame() for g in grains}
for inc in result.get(['F','P']).values():
    P = inc['P']
    F = inc['F']
    for g in grains:
        points = grid.material.flatten(order='F')==g
        P_11 = P[points,0,0].flatten()
        F_11 = np.broadcast_to(np.average(F[:,0,0]),P_11.shape)
        x = pd.DataFrame({'F_11':F_11,'P_11':P_11})
        data[g] = pd.concat((data[g],x),ignore_index=True)

In [ ]:
for g in grains:
   plot = sns.lineplot(y='P_11',x='F_11',data=data[g])

fig = plot.get_figure()

## some in-line visualization with pyvista

need to do the following environment setup  (use _virtual environment_ or _conda_ )
- `pip install 'pyvista[jupyter]>=0.38.1'` 
- `pip install ipywidgets 'pyvista[all,trame]'`
- `pip install trame`

_caution! there might be extra lib incompatibility!|_

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('trame')

In [ ]:
data_grid = pv.read(fn_vti)

pl = pv.Plotter()
pl.add_mesh(data_grid, scalars='material',edge_color='k',show_edges=True)
pl.add_axes()
pl.show()

In [ ]:
# get the last incr
rv = result.view(increments=-1)
rv.export_VTK()
last_inc = result.increments[-1]
fn_rv = fn_hdf.replace('.hdf5',f'_inc{last_inc}.vti')

In [ ]:
# plot sigma_vM of last incr
data_last = pv.read(fn_rv)

pl = pv.Plotter()
pl.add_mesh(data_last, scalars='phase/mechanical/sigma_vM / Pa', cmap='viridis')
pl.add_axes()
pl.show()